In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [39]:
def weighted_random_recommendation(items_weights, n=3):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    items_weights=items_weights.sort_values('weight', ascending=False).reset_index()
    return items_weights.loc[:n-1,:].sample(n)['item_id'].tolist()

In [40]:
items_weights = pd.DataFrame({'item_id':[1,2,3,4,5,6],'weight':[0.3,0.15,0.15,0.2,0.1,0.1]})
weighted_random_recommendation(items_weights)

[1, 4, 2]

### Задание 2. Расчет метрик
Рассчитайте Precision@3 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [97]:
result = pd.read_csv('predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[1124528, 14077738, 1125607, 10250749, 33883]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1081177, 894968]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[10356531, 1059930, 1038523, 259923, 8160230]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1081177, 894968]"


In [109]:
def precision_at_k(recommended_list, bought_list, k=3):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [110]:
# random_recommendation
import re
result.apply(lambda row: precision_at_k([int(x) for x in re.findall(r'\d+', row['random_recommendation'])], [int(x) for x in re.findall(r'\d+', row['actual'])]), axis=1).mean()

0.0004897159647404506

In [111]:
# popular_recommendation
result.apply(lambda row: precision_at_k([int(x) for x in re.findall(r'\d+', row['popular_recommendation'])], [int(x) for x in re.findall(r'\d+', row['actual'])]), axis=1).mean()


0.13777342474698007

In [112]:
# own_purchases
result.apply(lambda row: precision_at_k([int(x) for x in re.findall(r'\d+', row['own_purchases'])], [int(x) for x in re.findall(r'\d+', row['actual'])]), axis=1).mean()

0.03501469147894221